In [1]:
############################# Importing all req. libreries ############################################
import ipywidgets as ipw
from pymongo import MongoClient
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib
%matplotlib qt
import matplotlib.pyplot as plt
from bson.regex import Regex
from collections import Counter
import numpy as np

############################# Selecting the Entity IDs  ############################################
ids = pd.read_excel("IDs.xlsx")

Sup_id = ipw.Dropdown(
    options=list(ids["Supplier"].dropna()),
    description = "Supplier - ", )
typ = ipw.Dropdown(
    options=list(ids["Raw Material Type"].dropna()),
    description = "Raw mat Typ - ", )
ipw.HBox([Sup_id , typ])

In [2]:
Sup_id = Sup_id.value
typ = typ.value
client = MongoClient("mongodb://localhost:27017")
database = client["SCM_DB"]

############################# Printing the Selected Supplier details  ##################################
collection = database["Raw_Material_Colleciton"]
cruser = collection.find({"Supplier ID": Sup_id})
for doc in cruser: 
    doc_sup = doc["Supplier Location Details"]
    display(pd.Series(doc_sup,index=doc_sup.keys()))
    break
print("-------------------------------------------------------------------")
typ

Supplier_Name        Repuestos Samsung
Supplier_Address      Calle Quejio, 47
Supplier_City                  Seville
Supplier_Country                 Spain
Supplier_ZIP Code                41006
dtype: object

-------------------------------------------------------------------


'GPU'

In [3]:
############################# Printing the Selected Mobile Specifications  ##############################
collection = database["Retailer_orders_DB"]

pipeline_1 = [
    {
        u"$unwind": {
            u"path": u"$Order_Items_List"
        }
    }, 
    {
        u"$project": {
            u"time": u"$order_purchase_timestamp",
            u"Mobile_IMEI": u"$Order_Items_List",
            u"_id": 0.0
        }
    }, 
    {
        u"$out": u"temp"
    }
]

cursor = collection.aggregate(
    pipeline_1, 
    allowDiskUse = False
)
    
print("Temporary Collection created..")

Temporary Collection created..


In [4]:

collection = database["Raw_Material_Colleciton"]
pipeline_2 = [
    {
        u"$match": {
            u"Supplier ID": Sup_id,
            u"Raw Material IMEI number": {
                u"$regex": Regex(u".*"+str(typ)+".*")
            }
        }
    }, 
    {
        u"$unwind": {
            u"path": u"$Manufacturer IMEI Number"
        }
    }, 
    {
        u"$project": {
            u"_id": 0.0,
            u"Mobile_IMEI": u"$Manufacturer IMEI Number",
            u"Raw Material Description": u"$Raw Material Description",
            u"Avalibility": u"$Avalibility"
        }
    }, 
    {
        u"$lookup": {
            u"from": u"temp",
            u"localField": u"Mobile_IMEI",
            u"foreignField": u"Mobile_IMEI",
            u"as": u"k"
        }
    }, 
    {
        u"$unwind": {
            u"path": u"$k"
        }
    }, 
    {
        u"$project": {
            u"Raw Material Description": u"$Raw Material Description",
            u"Avalibility": u"$Avalibility",
            u"time": u"$k.time"
        }
    }, 
    {
        u"$group": {
            u"_id": u"$Raw Material Description",
            u"sold_on": {
                u"$push": u"$time"
            },
            u"Average_avalibility": {
                u"$avg": u"$Avalibility"
            }
        }
    }
]
cursor = collection.aggregate(
    pipeline_2, 
    allowDiskUse = False
)

def fun(doc):
    doc['sold_on'].sort()
    x = list(Counter(doc['sold_on']).keys())
    y = list(Counter(doc['sold_on']).values())
    label = "TYPE -- " + str(doc['_id']) + " -- Still in Stock " + str(round(doc['Average_avalibility'],2))
    plt.plot(x,y,"o-",label=label)
    plt.title('Details of "' + doc_sup["Supplier_Name"] + '" for "' + typ + '"' )
    plt.legend()
    plt.xticks(np.arange(5), ('Jun-19', 'Jul-19', 'Aug-19', 'Sep-19', 'Oct-19'))
    plt.show()
    
    
try:
    for doc in cursor:
        print(doc)
        print("-----------------------------------------------------------------------------------")
        fun(doc)
finally:
    client.close()


{'_id': 'Mali-T720MP2', 'sold_on': ['Jun-19', 'Jul-19', 'Oct-19', 'Jul-19', 'Sep-19', 'Oct-19', 'Jul-19', 'Oct-19', 'Oct-19', 'Oct-19', 'Jun-19', 'Aug-19', 'Sep-19', 'Oct-19', 'Jun-19', 'Jul-19', 'Jun-19', 'Jun-19', 'Oct-19', 'Jul-19', 'Oct-19', 'Aug-19', 'Aug-19', 'Jun-19', 'Jun-19', 'Jun-19', 'Jun-19', 'Jul-19', 'Oct-19', 'Jul-19', 'Oct-19', 'Aug-19', 'Oct-19', 'Jul-19', 'Jul-19', 'Jun-19', 'Aug-19', 'Sep-19', 'Oct-19', 'Jun-19', 'Aug-19', 'Aug-19', 'Aug-19', 'Jun-19', 'Jun-19', 'Sep-19', 'Jun-19', 'Sep-19', 'Aug-19', 'Jun-19', 'Jun-19', 'Aug-19', 'Aug-19', 'Oct-19', 'Sep-19', 'Jun-19', 'Jun-19', 'Jul-19', 'Oct-19', 'Jul-19', 'Sep-19', 'Jul-19', 'Oct-19', 'Jul-19', 'Jul-19'], 'Average_avalibility': 246.86153846153846}
-----------------------------------------------------------------------------------
{'_id': 'Adreno 505 - Sprint', 'sold_on': ['Aug-19', 'Oct-19', 'Oct-19', 'Jul-19', 'Jul-19', 'Jul-19', 'Oct-19', 'Aug-19', 'Jul-19', 'Jun-19', 'Aug-19', 'Jun-19', 'Jul-19', 'Jul-19', 'Ju